In [80]:
import pandas as pd
import numpy as np
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line, Timeline, Pie
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

In [81]:
# Part1 淘宝订单分析

# data loading
orders = pd.read_csv("OrderFromTmall/tmall_order_report.csv")
# data overview
orders.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [82]:
orders.head()

# remove white space
orders.columns = orders.columns.str.strip()

orders.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [83]:
# there are no duplicated rows
orders[orders.duplicated()].count()

订单编号        0
总金额         0
买家实际支付金额    0
收货地址        0
订单创建时间      0
订单付款时间      0
退款金额        0
dtype: int64

In [84]:
# 订单付款时间存在na值
orders.isna().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

In [85]:
# data cleaning
orders["收货地址"] = orders["收货地址"].str.replace('自治区|维吾尔|回族|壮族|省', '')
orders["收货地址"].unique()


array(['上海', '内蒙古自治区', '安徽省', '湖南省', '江苏省', '浙江省', '天津', '北京', '四川省',
       '贵州省', '辽宁省', '河南省', '广西壮族自治区', '广东省', '福建省', '海南省', '江西省', '甘肃省',
       '河北省', '黑龙江省', '云南省', '重庆', '山西省', '吉林省', '山东省', '陕西省', '湖北省',
       '青海省', '新疆维吾尔自治区', '宁夏回族自治区', '西藏自治区'], dtype=object)

In [86]:
rs = {}
rs["总订单数"] = orders["订单编号"].count()
rs["已完成订单数"] = orders["订单付款时间"].count()
rs["未付款订单数"] = orders["订单付款时间"].isna().sum()
rs["退款订单数"] = (orders["退款金额"]>0).sum()
rs["总订单金额"] = orders["总金额"].sum()
rs['总订单金额'] = orders['总金额'][orders['订单付款时间'].notnull()].sum() 
rs["总退款金额"] = orders["退款金额"][orders['订单付款时间'].notnull()].sum()
rs["总实际收入金额"] = orders["买家实际支付金额"][orders['订单付款时间'].notnull()].sum()
rs

{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总订单金额': 2474823.0700000003,
 '总退款金额': 572335.9199999999,
 '总实际收入金额': 1902487.1500000001}

In [87]:
# overview
table = Table()

headers = ['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '退款订单数', '总退款金额', '成交率', '退货率']

rows = [
    [
        rs["总订单数"], f"{rs['总订单金额']/10000:.2f} 万", rs["已完成订单数"], f"{rs["总实际收入金额"]/10000:.2f}万", 
        rs["退款订单数"], rs["总退款金额"], f"{rs["已完成订单数"]/rs["总订单数"]:.2%}", f"{rs["退款订单数"]/rs["总订单数"]:.2%}"
    ]
]

table.add(headers, rows)

table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况')
)

table.render_notebook()

总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48 万,24087,190.25万,5646,572335.9199999999,85.99%,20.16%


In [88]:
rs2 = orders.dropna().groupby('收货地址').agg({'订单编号':'count'})
rs2.nsmallest(n=5, columns="订单编号")
rs3 = rs2.to_dict()['订单编号']
c = (
    Map()
    .add("订单量", [*rs3.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='地区分布'),
        visualmap_opts=opts.VisualMapOpts(max_=1000),            
    )
)

c.render_notebook()

In [89]:
orders['订单创建时间'] = pd.to_datetime(orders['订单创建时间'])
orders['订单付款时间'] = pd.to_datetime(orders['订单付款时间'])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   订单编号      28010 non-null  int64         
 1   总金额       28010 non-null  float64       
 2   买家实际支付金额  28010 non-null  float64       
 3   收货地址      28010 non-null  object        
 4   订单创建时间    28010 non-null  datetime64[ns]
 5   订单付款时间    24087 non-null  datetime64[ns]
 6   退款金额      28010 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(1)
memory usage: 1.5+ MB


In [90]:
rs4 = orders.groupby(orders["订单创建时间"].apply(lambda x:x.strftime("%Y-%m-%d"))).agg({"订单编号":"count"})
dict = rs4.to_dict()["订单编号"]
x_values = dict.keys()
y_values = dict.values()

c = (
    Line()
    .add_xaxis(list(x_values))
    .add_yaxis("订单量", list(y_values))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每日订单量走势"))
)

c.render_notebook()

In [91]:
# 从每小时订单量走势来看，一天中有3个高峰期（10点、15点、21点）
rs5 = orders.groupby(orders["订单创建时间"].apply(lambda x:x.strftime("%H"))).agg({"订单编号":"count"})

dict = rs5.to_dict()["订单编号"]

x= [*dict.keys()]
y= [*dict.values()]



c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("订单量", y)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每小时订单量走势"))
)

c.render_notebook()

In [92]:
# average payment time
s = orders['订单付款时间'] - orders['订单创建时间']
s.dropna().apply(lambda x: x.seconds/60).mean()

7.7399046511949745

In [93]:
# Part2 双十一美妆数据分析

# overview
cosmetics_sales =  pd.read_csv("OrderFromTmall\双十一淘宝美妆数据.csv")
cosmetics_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27598 non-null  object 
 1   id             27598 non-null  object 
 2   title          27598 non-null  object 
 3   price          27598 non-null  float64
 4   sale_count     25244 non-null  float64
 5   comment_count  25244 non-null  float64
 6   店名             27598 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


In [94]:
cosmetics_sales.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2016/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2016/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2016/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2016/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2016/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [95]:
# check dup rows
(cosmetics_sales.duplicated()).sum()

cosmetics_sales.drop_duplicates(inplace=True)


In [96]:
cosmetics_sales.isna().sum()
cosmetics_sales.fillna(0, inplace=True)

In [97]:
cosmetics_sales.reset_index(drop=True, inplace=True)

In [98]:
# formatting datetime
cosmetics_sales["update_time"] = pd.to_datetime(cosmetics_sales["update_time"]).apply(lambda x : x.strftime("%Y-%m-%d"))

In [99]:
# add sales_amount column
cosmetics_sales["sales_amount"] = cosmetics_sales["price"] * cosmetics_sales["sale_count"]
cosmetics_sales[cosmetics_sales["sales_amount"]>0].sort_values("sales_amount")

,update_time,id,title,price,sale_count,comment_count,店名,sales_amount
27042,2016-11-05,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
26530,2016-11-07,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
26785,2016-11-06,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
26273,2016-11-08,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
26018,2016-11-09,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,2.0,0.0,佰草集,2.0
...,...,...,...,...,...,...,...,...
10439,2016-11-09,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433603.0,9823.0,相宜本草,94525454.0
10306,2016-11-10,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433604.0,9712.0,相宜本草,94525672.0
10027,2016-11-13,A13956982789,相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,444263.0,9427.0,相宜本草,96849334.0
9940,2016-11-14,A13956982789,相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,444271.0,9355.0,相宜本草,96851078.0


In [100]:
# 每日销售量走势
rs1 =  cosmetics_sales.groupby("update_time").agg({"sale_count": "sum"})

dict = rs1.to_dict()["sale_count"]
x_values = [*dict.keys()]
y_values = [*dict.values()]

c = (
    Line()
    .add_xaxis(x_values)
    .add_yaxis("销售量", y_values)
    .set_series_opts(
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值"),
                opts.MarkPointItem(type_="average", name="平均值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每日整体销售量走势"))
)

c.render_notebook()

In [101]:

dates = list(cosmetics_sales["update_time"].unique())
dates.reverse()


In [102]:
# 品牌销售量
tl = Timeline()
tl.add_schema(
#         is_auto_play=True,
        is_loop_play=False,
        play_interval=500,
    )


for date in dates:
    items = cosmetics_sales[cosmetics_sales["update_time"] <= date].groupby("店名").agg(
        {"sales_amount": "sum", "sale_count": "sum"}).sort_values(by="sale_count", ascending=False)[:10].sort_values(by="sales_amount", ascending=False).to_dict()
    bar = (
        Bar()
        .add_xaxis([*items['sale_count'].keys()])
        .add_yaxis("销售量", [round(val/10000, 2) for val in items['sale_count'].values()], label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} 万'))
        .add_yaxis("销售额", [round(val/100000000, 2) for val in items['sales_amount'].values()], label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} 亿元'))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts("累计销售量排行 TOP10")
        )
    )

    tl.add(bar, date)

tl.render_notebook()

In [103]:
dict = cosmetics_sales.groupby("店名").agg({"sale_count":"sum"}).sort_values(by="sale_count",ascending=False)[:10].to_dict()["sale_count"]

# dict

pie = (
    Pie().
    add("销量",[*dict.items()]).
    set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} 万({d}%)"))
)

pie.render_notebook()

In [104]:
# 品牌价格比较
dict = cosmetics_sales.groupby("店名").agg({"price":"mean"}).sort_values(by="price", ascending=False)[:10].sort_values(by="price").to_dict()["price"]
dict = {k:round(v,2) for k,v in dict.items()}
x_values = [*dict.keys()]
y_values = [*dict.values()]

bar = (
    Bar()
    .add_xaxis(x_values)
    .add_yaxis("平均价格", y_values)
    .set_series_opts(
        label_opts=opts.LabelOpts(position="right"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="品牌平均价格"))
    .reversal_axis()
)

bar.render_notebook()


In [105]:
# Part3 日化分析

sales =  pd.read_excel("OrderFromTmall\日化.xlsx", sheet_name="销售订单表")
prod_info =  pd.read_excel("OrderFromTmall\日化.xlsx", sheet_name="商品信息表")
# chemicals.info()
prod_info.info()
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   商品编号    122 non-null    object
 1   商品名称    122 non-null    object
 2   商品小类    122 non-null    object
 3   商品大类    122 non-null    object
 4   销售单价    122 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31452 entries, 0 to 31451
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   订单编码    31452 non-null  object 
 1   订单日期    31452 non-null  object 
 2   客户编码    31452 non-null  object 
 3   所在区域    31450 non-null  object 
 4   所在省份    31450 non-null  object 
 5   所在地市    31452 non-null  object 
 6   商品编号    31451 non-null  object 
 7   订购数量    31450 non-null  object 
 8   订购单价    31448 non-null  object 
 9   金额      31448 non-null  float64
dtypes: float64(1), object(9)
memory usag

In [106]:
# data cleaning for prod_info
(prod_info.duplicated()).sum()
(prod_info.isna()).sum()

商品编号    0
商品名称    0
商品小类    0
商品大类    0
销售单价    0
dtype: int64

In [107]:
# data cleaning for sales

(sales.duplicated()).sum()

sales.drop_duplicates(inplace=True)

sales.fillna(method="ffill", inplace=True)
sales.fillna(method="bfill", inplace=True)

(sales.isna()).sum()

sales.reset_index(drop=True, inplace=True)

C:\Users\13097\AppData\Local\Temp\ipykernel_27068\75574457.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales.fillna(method="ffill", inplace=True)
C:\Users\13097\AppData\Local\Temp\ipykernel_27068\75574457.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales.fillna(method="bfill", inplace=True)


In [108]:
sales['订单日期'] = sales['订单日期'].apply(lambda x: pd.to_datetime(x, format='%Y#%m#%d') if isinstance(x, str) else x)

# filter dirty data
sales = sales[sales["订单日期"] < "2024-03-28"]

sales["订单日期"].min(), sales["订单日期"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2019-09-30 00:00:00'))

In [117]:
sales["订购数量"] = sales["订购数量"].apply(lambda x : x.strip("个") if isinstance(x, str) else x).astype("int")
sales["订购单价"] = sales["订购单价"].apply(lambda x : x.strip("元") if isinstance(x, str) else x).astype("float")
sales["金额"] = sales["金额"].astype("float")
sales.info()


<class 'pandas.core.frame.DataFrame'>
Index: 31445 entries, 0 to 31445
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   订单编码    31445 non-null  object        
 1   订单日期    31445 non-null  datetime64[ns]
 2   客户编码    31445 non-null  object        
 3   所在区域    31445 non-null  object        
 4   所在省份    31445 non-null  object        
 5   所在地市    31445 non-null  object        
 6   商品编号    31445 non-null  object        
 7   订购数量    31445 non-null  int32         
 8   订购单价    31445 non-null  float64       
 9   金额      31445 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(6)
memory usage: 2.5+ MB


In [110]:
sales["所在省份"] = sales["所在省份"].str.replace('自治区|维吾尔|回族|壮族|省|市', '')
sales["客户编码"] = sales["客户编码"].str.replace('编号','')
# sales["商品编号"].str.contains("编号").sum()
sales.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,214.0,190888.0
1,D21329,2019-05-14,S11460,东区,安徽省,宿州市,X005,276,185.0,51060.0
2,D22372,2019-08-26,S11101,北区,山西省,忻州市,X078,1450,116.0,168200.0
3,D31078,2019-04-08,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102.0,187068.0
4,D32470,2019-04-11,S18696,北区,北京市,北京市,X010,887,58.0,51446.0


In [111]:
sales.head()
dict = sales.groupby(sales["订单日期"].dt.month).agg(
    {"订购数量": "sum", "金额": "sum"}).to_dict()

x_values = [*dict["订购数量"].keys()]
y_values_1 = [*dict["订购数量"].values()]
y_values_2 = [*dict["金额"].values()]

bar = (
    Bar()
    .add_yaxis("订购数量", [round(i/10000, 2) for i in y_values_1], label_opts=opts.LabelOpts(position="top", formatter='{@[1]/} 万'))
    .add_yaxis("金额", [round(i/100000000, 2) for i in y_values_2], label_opts=opts.LabelOpts(position="top", formatter='{@[1]/} 亿'))
    .add_xaxis([f"{m}月" for m in x_values])
    .set_series_opts(
        markpoint_opts=opts.MarkPointOpts(
            data=[
                # opts.MarkPointItem(type_="max", name="最大值"),
                # opts.MarkPointItem(type_="min", name="最小值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每月销售情况"))
)

bar.render_notebook()

In [112]:
# 地区销量分布
dict = sales.groupby("所在地市").agg({"订购数量": "sum"}).sort_values("订购数量", ascending=False)[:10].sort_values("订购数量").to_dict()

x_values = [*dict["订购数量"].keys()]
y_values = [*dict["订购数量"].values()]

bar = (
    Bar()
    .add_xaxis(x_values)
    .add_yaxis("订购数量", [round(i/10000,2) for i in y_values], label_opts=opts.LabelOpts(position="right",formatter='{@[1]/} 万'))
    .reversal_axis()
    .set_global_opts(title_opts=opts.TitleOpts(title="地区销量"))
)

bar.render_notebook()


In [113]:
# 日化类型需求
sales.head()

df = pd.merge(sales,prod_info,on="商品编号")

df.head()



,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额,商品名称,商品小类,商品大类,销售单价
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,214.0,190888.0,商品91,粉底,彩妆,214
1,D26674,2019-05-01,S15128,东区,江苏省,南通市,X091,1133,214.0,242462.0,商品91,粉底,彩妆,214
2,D23381,2019-09-22,S17133,东区,江苏省,宿迁市,X091,1136,214.0,243104.0,商品91,粉底,彩妆,214
3,D29060,2019-09-10,S14106,东区,江苏省,常州市,X091,544,214.0,116416.0,商品91,粉底,彩妆,214
4,D21234,2019-07-03,S17197,东区,湖北省,十堰市,X091,342,214.0,73188.0,商品91,粉底,彩妆,214


In [114]:
df.groupby(["商品大类","商品小类"]).agg({"订购数量":"sum"}).sort_values("订购数量", ascending=False)


订购数量
商品大类 商品小类         
护肤品  面膜    5451914
     面霜    4567570
     爽肤水   3523687
     眼霜    3350078
     隔离霜   2488124
     防晒霜   2388610
彩妆   口红    2013024
护肤品  洁面乳   1928020
彩妆   粉底    1188621
     睫毛膏    587399
     眼影     296599
     蜜粉      45534

In [115]:
# 销量地区分布
sales.head()

dict = sales.groupby("所在省份").agg({"订购数量":"sum"}).to_dict()['订购数量']

map = (
    Map()
    .add("订购数量", [*dict.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='省份分布'),
        visualmap_opts=opts.VisualMapOpts(max_=1000000),            
    )
)

map.render_notebook()

In [116]:
# 通过RFM模型挖掘客户价值
# weight: Recency:20, Frequency:30, Money:50

df = sales.groupby("客户编码").agg({"订单日期": "max", "订单编码": "count", "金额": "sum"})
df.columns = ["R", "F", "M"]
df["r"] = df["R"].rank(pct=True)
df["f"] = df["F"].rank(pct=True)
df["m"] = df["M"].rank(pct=True)
df["score"] = df["r"] * 20 + df["f"] * 30 + df["m"] * 50
df["score"] = df["score"].round(2)
df =  df.sort_values(by="score", ascending=False)
df["class"] = np.where((df["score"]>85),"一级","二级")
df

,R,F,M,r,f,m,score,class
客户编码,,,,,,,,
S17476,2019-09-30,69,10325832.0,0.980148,0.986611,0.987073,98.55,一级
S22326,2019-09-30,62,10074609.0,0.980148,0.973223,0.984303,98.01,一级
S11581,2019-09-28,79,10333668.0,0.918283,0.996768,0.987996,97.67,一级
S12848,2019-09-29,66,9673572.0,0.944598,0.980609,0.980609,97.34,一级
S19095,2019-09-26,81,11031632.0,0.864728,0.999077,0.996307,97.08,一级
...,...,...,...,...,...,...,...,...
S12690,2019-05-07,7,917233.0,0.012927,0.022622,0.024931,2.18,二级
S11176,2019-06-09,7,614134.0,0.036011,0.022622,0.009234,1.86,二级
S18379,2019-07-05,4,400195.0,0.071099,0.003232,0.004617,1.75,二级
